<a href="https://colab.research.google.com/github/Brianne-Bell/ADS-505/blob/main/Project_Fraud_eCommerce_BB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ADS-505 Project: Fraud E-Commerce
> https://www.kaggle.com/datasets/vbinh002/fraud-ecommerce?resource=download
## Team 3
> Dave Friesen
>
> Brianne Bell
>
> Michael Nguyen
## Importing Required Packages

In [ ]:
import pandas as pd
import numpy as np  
import seaborn as sb
import datetime

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression
import matplotlib.pylab as plt
from sklearn import preprocessing
import statsmodels.formula.api as sm
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

import dmba
from dmba import regressionSummary, plotDecisionTree
from dmba import adjusted_r2_score, AIC_score, BIC_score
from dmba import backward_elimination, forward_selection, stepwise_selection
from dmba import gainsChart, liftChart
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

%matplotlib inline

## Dataset(s) Loading and Information
### Fraud e-commerce: 'Fraud_Data' and 'IpAddress_to_Country'

In [ ]:
# Loading the Fraud_Data.csv
fraudorig_df = pd.read_csv(
    'C:/Users/breel.B-E-BELL/OneDrive/Documents/USD_MastersAppliedDataScience/ADS-505 ADSforBusiness/Final Project Files/Fraud_Data.csv')
print(fraudorig_df.shape)
    # 11 columns and 151,112 entries
print(fraudorig_df.info())
    # no null datapoints

(151112, 11)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151112 entries, 0 to 151111
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   user_id         151112 non-null  int64  
 1   signup_time     151112 non-null  object 
 2   purchase_time   151112 non-null  object 
 3   purchase_value  151112 non-null  int64  
 4   device_id       151112 non-null  object 
 5   source          151112 non-null  object 
 6   browser         151112 non-null  object 
 7   sex             151112 non-null  object 
 8   age             151112 non-null  int64  
 9   ip_address      151112 non-null  float64
 10  class           151112 non-null  int64  
dtypes: float64(1), int64(4), object(6)
memory usage: 12.7+ MB
None


In [ ]:
fraudorig_df.head(5)
fraudclass_zero = round(((fraudorig_df['class'].value_counts()[0])/len(fraudorig_df))*100,2)
fraudclass_one = round(((fraudorig_df['class'].value_counts()[1])/len(fraudorig_df))*100,2)
print('% fraud class 0: ', fraudclass_zero, 'no fraud') 
print('% fraud class 1: ', fraudclass_one, 'fraud detected')

% fraud class 0:  90.64 no fraud
% fraud class 1:  9.36 fraud detected


In [ ]:
# Loading the Fraud_Data.csv
ip_df = pd.read_csv(
    'C:/Users/breel.B-E-BELL/OneDrive/Documents/USD_MastersAppliedDataScience/ADS-505 ADSforBusiness/Final Project Files/IpAddress_to_Country.csv',)
print(ip_df.shape)
    # 3 columns and 138,846 entries
print(ip_df.info())
    # no null datapoints
print('\n', 'Number of Unique Entries', '\n', ip_df.nunique())

(138846, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138846 entries, 0 to 138845
Data columns (total 3 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   lower_bound_ip_address  138846 non-null  float64
 1   upper_bound_ip_address  138846 non-null  int64  
 2   country                 138846 non-null  object 
dtypes: float64(1), int64(1), object(1)
memory usage: 3.2+ MB
None

 Number of Unique Entries 
 lower_bound_ip_address    138846
upper_bound_ip_address    138846
country                      235
dtype: int64


### Combining dataframes where IP in fraud_df is within bounds of the IP bounds of ip_df to get Country name

In [ ]:
# making columns in question float type
ip_df['lower_bound_ip_address'] = ip_df['lower_bound_ip_address'].astype('float') # technically already is float
ip_df['upper_bound_ip_address'] = ip_df['upper_bound_ip_address'].astype('float') # integer, needs to be float
fraudorig_df['ip_address'] = fraudorig_df['ip_address'].astype('float') # tecnically already is float
# chekcing
print(ip_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138846 entries, 0 to 138845
Data columns (total 3 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   lower_bound_ip_address  138846 non-null  float64
 1   upper_bound_ip_address  138846 non-null  float64
 2   country                 138846 non-null  object 
dtypes: float64(2), object(1)
memory usage: 3.2+ MB
None


In [ ]:
fraud_df = fraudorig_df.copy()
fraud_df.head(5)

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0


### Filling in Country values 
If the IP address doesn't fit into the bounds of the ip dataframe then it is assumed they are from VPN users.

In [ ]:
# function takes in ip_address (fraud_df) and compares it to upper/lower bounds in ip_df to return country name
def ip_to_country(ip):
    try:
        return ip_df.country[
            (ip_df.lower_bound_ip_address < ip) &
            (ip_df.upper_bound_ip_address > ip)].iloc[0]
    except IndexError:
        return 'VPNuser'
    
# calling the function
fraud_df['ipCountry'] = fraudorig_df['ip_address'].apply(ip_to_country)

# saving to csv file to speed up when opening notebook next time
fraud_df.to_csv('C:/Users/breel.B-E-BELL/OneDrive/Documents/USD_MastersAppliedDataScience/ADS-505 ADSforBusiness/Final Project Files/Fraud_plus_country.csv')

# checking
fraud_df.head(5)

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,ipCountry
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0,Japan
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0,United States
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1,United States
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0,VPNuser
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0,United States


In [ ]:
fraud_df = pd.read_csv(
    'C:/Users/breel.B-E-BELL/OneDrive/Documents/USD_MastersAppliedDataScience/ADS-505 ADSforBusiness/Final Project Files/Fraud_plus_country.csv')

In [ ]:
# seeing how many were unknown
print('There were %i unknown Country values (likely using a VPN service)' %fraud_df['ipCountry'].value_counts()['VPNuser'])
print('This is %.1f percent of the data' %(fraud_df['ipCountry'].value_counts()['VPNuser']/len(fraud_df)*100))


There were 21966 unknown Country values (likely using a VPN service)
This is 14.5 percent of the data


### Datetime Manipulations

In [ ]:
# calculating time difference between signing up and purchasing
fraud_df['time_diff_hr'] = (pd.to_datetime(fraud_df['purchase_time'])
                                            - pd.to_datetime(fraud_df['signup_time'])).astype('timedelta64[h]')

fraud_df.head(5)

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,ipCountry,time_diff_hr
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0,Japan,1251.0
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0,United States,4.0
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1,United States,0.0
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0,VPNuser,136.0
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0,United States,1211.0


In [ ]:
# renaming
df = fraud_df.copy()
df = df.rename(columns= {'signup_time':'signup', 'purchase_time':'purchase', 'ipCountry':'ip_country'})
df = df.drop(['user_id'], axis=1)

In [ ]:
# converting from_date to datetime (date and time separate columns) instead of string
df['signup_date'] = pd.to_datetime(df['signup']).dt.strftime('%m/%d/%y')
df['signup_time'] = pd.to_datetime(df['signup']).dt.strftime('%H:%M:%S')

# converting booking_created to datetime (date and time separate columns) instead of string
df['purchase_date'] = pd.to_datetime(df['purchase']).dt.strftime('%m/%d/%y')
df['purchase_time'] = pd.to_datetime(df['purchase']).dt.strftime('%H:%M:%S')


#checking
df.head(3)

,signup,purchase,purchase_value,device_id,source,browser,sex,age,ip_address,class,ip_country,time_diff_hr,signup_date,signup_time,purchase_date,purchase_time
0,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0,Japan,1251.0,02/24/15,22:55:49,04/18/15,02:47:11
1,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0,United States,4.0,06/07/15,20:39:50,06/08/15,01:38:54
2,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1,United States,0.0,01/01/15,18:52:44,01/01/15,18:52:45


In [ ]:
# making date a month instead of full date
df['signup_month'] = pd.to_datetime(df['signup']).dt.month_name()
df['purchase_month'] = pd.to_datetime(df['purchase']).dt.month_name()

# checking, can comment out
# df.head(2)

In [ ]:
# looking days of week
df['signup_day'] = pd.to_datetime(df['signup_date']).dt.day_name()
df['purchase_day'] = pd.to_datetime(df['purchase_date']).dt.day_name()

# zooming in on just weekend (saturday, sunday) or weekday (monday-friday)
    # np.where(condition, when true it's this, when not it's this)
df['signup_weektype'] = np.where((df['signup_day']=='Saturday') | #first condition followed by | (or)
                                   (df['signup_day']=='Sunday'), # second condition
                                  'Weekend', 'Weekday') # if true response, if false response

df['purchase_weektype'] = np.where((df['purchase_day']=='Saturday') | #first condition followed by | (or)
                                   (df['purchase_day']=='Sunday'), # second condition
                                  'Weekend', 'Weekday') # if true response, if false response
# checking
df.head(5)

,signup,purchase,purchase_value,device_id,source,browser,sex,age,ip_address,class,...,signup_date,signup_time,purchase_date,purchase_time,signup_month,purchase_month,signup_day,purchase_day,signup_weektype,purchase_weektype
0,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0,...,02/24/15,22:55:49,04/18/15,02:47:11,February,April,Tuesday,Saturday,Weekday,Weekend
1,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0,...,06/07/15,20:39:50,06/08/15,01:38:54,June,June,Sunday,Monday,Weekend,Weekday
2,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1,...,01/01/15,18:52:44,01/01/15,18:52:45,January,January,Thursday,Thursday,Weekday,Weekday
3,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0,...,04/28/15,21:13:25,05/04/15,13:54:50,April,May,Tuesday,Monday,Weekday,Weekday
4,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0,...,07/21/15,07:09:52,09/09/15,18:40:53,July,September,Tuesday,Wednesday,Weekday,Weekday


In [ ]:
# splitting time into categories instead of times for leaving data
# AM: 0-11, PM: 12-23
sign_timewise = [(pd.to_datetime(df['signup_time']).dt.hour.between(0, 11)), # am
           (pd.to_datetime(df['signup_time']).dt.hour.between(12, 23)), # pm
           ]
time_values = ['AM', 'PM']

# bringing the splitting into the df3 via np.select(timewise, time_values):
df['signup_timegroup'] = np.select(sign_timewise, time_values)

# purchase
purch_timewise = [(pd.to_datetime(df['purchase_time']).dt.hour.between(0, 11)), # am
           (pd.to_datetime(df['purchase_time']).dt.hour.between(12, 23)), # pm
           ]


# bringing the splitting into the df3 via np.select(timewise, time_values):
df['purchase_timegroup'] = np.select(purch_timewise, time_values)

#checking
df.head(3)

,signup,purchase,purchase_value,device_id,source,browser,sex,age,ip_address,class,...,purchase_date,purchase_time,signup_month,purchase_month,signup_day,purchase_day,signup_weektype,purchase_weektype,signup_timegroup,purchase_timegroup
0,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0,...,04/18/15,02:47:11,February,April,Tuesday,Saturday,Weekday,Weekend,PM,AM
1,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0,...,06/08/15,01:38:54,June,June,Sunday,Monday,Weekend,Weekday,PM,AM
2,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1,...,01/01/15,18:52:45,January,January,Thursday,Thursday,Weekday,Weekday,PM,PM


In [ ]:
# dropping signup, purchase, signup_date, signup_time, purchase_date, purchase_time, ip_address columns
df = df.drop(['signup', 'purchase', 'signup_date', 'signup_time', 'purchase_date', 'purchase_time', 'ip_address'], axis=1)

# we can drop the day of the week one if we'd like so then we just have weekday vs weekend

# checking
df.head(3)

,purchase_value,device_id,source,browser,sex,age,class,ip_country,time_diff_hr,signup_month,purchase_month,signup_day,purchase_day,signup_weektype,purchase_weektype,signup_timegroup,purchase_timegroup
0,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,0,Japan,1251.0,February,April,Tuesday,Saturday,Weekday,Weekend,PM,AM
1,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,0,United States,4.0,June,June,Sunday,Monday,Weekend,Weekday,PM,AM
2,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,1,United States,0.0,January,January,Thursday,Thursday,Weekday,Weekday,PM,PM


### Categorical columns treated with get_dummies

In [ ]:
# !pip install pycountry_convert

  Installing build dependencies: started
  Installing build dependencies: still running...
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for pycountry: filename=pycountry-22.3.5-py2.py3-none-any.whl size=10681832 sha256=fdbf18ccf90806dc21bf971222281a1117d15cc9499939e17ae513fe186d552b
  Stored in directory: c:\users\breel.b-e-bell\appdata\local\pip\cache\wheels\e2\aa\0f\c224e473b464387170b83ca7c66947b4a7e33e8d903a679748
Successfully built pycountry


In [ ]:
df.shape

(151111, 17)

In [ ]:
# "Invalid Country Name: 'Bonaire; Sint Eustatius; Saba'" from running country conversion code so dropping that one instance
df.drop(df[df['ip_country']=='Bonaire; Sint Eustatius; Saba'].index, inplace=True)
df.drop(df[df['ip_country']=='Reunion'].index, inplace=True)
df.shape

In [ ]:
# making a copy of dataframe in case it goes weird
df2 = df.copy()

In [ ]:
# changing formatting for troublesome names
df2.loc[df2['ip_country']=='Korea Republic of', 'ip_country'] = 'Korea, Republic of'

df2.loc[df2['ip_country']=='Taiwan; Republic of China (ROC)', 'ip_country'] = 'Taiwan'

df2.loc[df2['ip_country']=='Iran (ISLAMIC Republic Of)', 'ip_country'] = 'Iran'

df2.loc[df2['ip_country']=='Moldova Republic of', 'ip_country'] = 'Moldova, Republic of'

df2.loc[df2['ip_country']=='Croatia (LOCAL Name: Hrvatska)', 'ip_country'] = 'Croatia'

df2.loc[df2['ip_country']=='Slovakia (SLOVAK Republic)', 'ip_country'] = 'Slovakia'

df2.loc[df2['ip_country']=="Cote D'ivoire", 'ip_country'] = 'Ivory Coast'

df2.loc[df2['ip_country']=='Virgin Islands (U.S.)', 'ip_country'] = 'United States'

df2.loc[df2['ip_country']=='Tanzania United Republic of', 'ip_country'] = 'Tanzania, United Republic of'

df2.loc[df2['ip_country']=='Congo The Democratic Republic of The', 'ip_country'] = 'Congo'

In [ ]:
# seeing how many countries there are
df2['ip_country'].unique()

# there's a lot, especially if we use get_dummies on it. 
## grouping into continents/area groupings

import pycountry_convert as pc

def country_to_continent(country_name):
    if country_name == 'VPNuser':
        return 'VPNuser'
    if country_name == 'European Union':
        return 'Europe'
    if country_name == 'Libyan Arab Jamahiriya':
        return 'Africa'
    if country_name == 'Bosnia and Herzegowina':
        return 'Europe'
    if country_name == 'Palestinian Territory Occupied':
        return 'Africa'
    if country_name == 'Curacao':
        return 'South America'
    country_alpha2 = pc.country_name_to_country_alpha2(country_name)
    country_continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
    country_continent_name = pc.convert_continent_code_to_continent_name(country_continent_code)
    
    return country_continent_name

# calling the function
df2['continent'] = df2['ip_country'].apply(lambda x: country_to_continent(x))

#checking
df2[['continent', 'ip_country']].head(5)

,continent,ip_country
0,Asia,Japan
1,North America,United States
2,North America,United States
3,VPNuser,VPNuser
4,North America,United States


In [ ]:
# checking for unique continents (should be no surprises)
df2['continent'].unique()

array(['Asia', 'North America', 'VPNuser', 'South America', 'Europe',
       'Africa', 'Oceania'], dtype=object)

In [ ]:
# marking categorical columns (browser, source, sex, signup_month, purchase_month, 
# signup_day, purchase_day, signup_weektype, purchase_weektype, signup_timegroup, purchase_timegroup)
df2.browser = df2.browser.astype('category')
df2.source = df2.source.astype('category')
df2.sex = df2.sex.astype('category')
df2.continent = df2.continent.astype('category')
df2.signup_month = df2.signup_month.astype('category')
df2.purchase_month = df2.purchase_month.astype('category')
df2.signup_day = df2.signup_day.astype('category')
df2.purchase_day = df2.purchase_day.astype('category')
df2.signup_weektype = df2.signup_weektype.astype('category')
df2.purchase_weektype = df2.purchase_weektype.astype('category')
df2.signup_timegroup = df2.signup_timegroup.astype('category')
df2.purchase_timegroup = df2.purchase_timegroup.astype('category')

# checking
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 151111 entries, 0 to 151111
Data columns (total 17 columns):
 #   Column              Non-Null Count   Dtype   
---  ------              --------------   -----   
 0   purchase_value      151111 non-null  int64   
 1   device_id           151111 non-null  object  
 2   source              151111 non-null  category
 3   browser             151111 non-null  category
 4   sex                 151111 non-null  category
 5   age                 151111 non-null  int64   
 6   class               151111 non-null  int64   
 7   ip_country          151111 non-null  object  
 8   time_diff_hr        151111 non-null  float64 
 9   signup_month        151111 non-null  object  
 10  purchase_month      151111 non-null  object  
 11  signup_day          151111 non-null  object  
 12  purchase_day        151111 non-null  object  
 13  signup_weektype     151111 non-null  object  
 14  purchase_weektype   151111 non-null  object  
 15  signup_timegroup 

In [ ]:
# listing categorical columns to keep object ones (device_id) from being included
cats = ['browser', 'source', 'sex', 'continent', 'signup_month', 'purchase_month', 
        'signup_day', 'purchase_day', 'signup_weektype', 'purchase_weektype', 
        'signup_timegroup', 'purchase_timegroup']

# getting dummy variables for categorical values
df_dummies = pd.get_dummies(df, drop_first=True) # translate category variables into binary
df_dummies.head()

,purchase_value,age,class,time_diff_hr,device_id_AAAWIHVCQELTP,device_id_AAAXJHWCLISKY,device_id_AAAXXOZJRZRAO,device_id_AABFGRPBQHWFQ,device_id_AABGCAPIYUWNC,device_id_AABJEESQPJHDQ,...,purchase_day_Monday,purchase_day_Saturday,purchase_day_Sunday,purchase_day_Thursday,purchase_day_Tuesday,purchase_day_Wednesday,signup_weektype_Weekend,purchase_weektype_Weekend,signup_timegroup_PM,purchase_timegroup_PM
0,34,39,0,1251.0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,1,0
1,16,53,0,4.0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,1,0
2,15,53,1,0.0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,1
3,44,41,0,136.0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,1
4,39,45,0,1211.0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
